---

# Data Quality Assessment Package Demo

Import packages

In [ ]:
from SKA_assignment.data_handler import DataHandler
from SKA_assignment.analysing import monitor_data_quality_over_time, visualize_data
from SKA_assignment.plotting import plot_amplitude_vs_time
from SKA_assignment.utils import get_combined_masks, get_binned_visibility_amplitude

Read in the data

In [ ]:
data_handler = DataHandler("pipeline_problem_data.ms")

Image the 'unprocessed' data

In [ ]:
# Plot all data in one dirty and one clean image.
visualize_data(data_handler, first_t_frame=0, nb_t_steps=1, t_step=120, first_freq_step=0, n_freq_steps=1, freq_step=32, vmin=-20000, vmax=200000)

There seems to be an interesting feature on the sky, perhaps a new SKAO supernova remnant?

Let's try to see it clearer. Perhaps it is brighter in a subset of the band?

In [ ]:
# Plot each frequency channel in a separate set of dirty and clean images.
visualize_data(data_handler, first_t_frame=0, nb_t_steps=1, t_step=120, first_freq_step=0, n_freq_steps=32, freq_step=1, vmin=-20000, vmax=20000)

Channel 30 looks a bit suspect and would warant extra investigation, but no obvious signature of the SKAO feature in only a subset of channels.

Perhaps it is a transient signal?

In [ ]:
# Plot a subset of time frames in a separate set of dirty and clean images.
visualize_data(data_handler, first_t_frame=45, nb_t_steps=10, t_step=1, first_freq_step=0, n_freq_steps=1, freq_step=32, vmin=-20000, vmax=20000)

Interesting, SKAO shines bright in the 48th timeframe.

There's also something funny happening in the 50th timeframe.

Let's look at a plot of average visibility amplitude over time

In [ ]:
first_freq_step = 0
n_freq_steps = 32
freq_step = 1
first_t_frame = 0
nb_t_steps = 120
t_step = 1

# Unpack data from the DataHandler
time_all = data_handler.time_all
autocorr_filter = data_handler.get_autocorr_filter()
vis = data_handler.get_visibilities()
unique_times = data_handler.get_times()
dt = data_handler.get_time_step()

combined_masks = get_combined_masks(
    time_all, unique_times, dt, autocorr_filter, first_t_frame, nb_t_steps, t_step
)
binned_amplitude = get_binned_visibility_amplitude(
    vis, combined_masks, first_freq_step, n_freq_steps, freq_step
)

# Plot results for each frequency
for i in range(n_freq_steps):
    title = f"Time Series of Visibility Amplitude for Channel {first_freq_step + i * freq_step}"
    filename = (
        f"amplitude_vs_time_channel_{first_freq_step + i * freq_step}.png"
    )
    plot_amplitude_vs_time(
        binned_amplitude[i, :],
        range(first_t_frame, first_t_frame + nb_t_steps),
        title,
        filename,
        save=False
    )

A lot of interesting features!

Let's see if we can remove the systematic peaks and the SKAO peak.

In [ ]:
outliers = monitor_data_quality_over_time(data_handler, generate_plots=True, save_plots=False, flag_multiplier=10)

As a reminder, this is where we started from

In [ ]:
# Plot all data in one dirty and one clean image.
visualize_data(data_handler, first_t_frame=0, nb_t_steps=1, t_step=120, first_freq_step=0, n_freq_steps=1, freq_step=32, vmin=-20000, vmax=200000)

So how does this do? Let's image the visibilities without the flagged outliers

In [ ]:
visualize_data(data_handler, flags=outliers, first_t_frame=0, nb_t_steps=1, t_step=120, first_freq_step=0, n_freq_steps=1, freq_step=32, vmin=-20000, vmax=200000)

Many more things to do:
- Analysis:
    - Plot visibilities per antenna
    - Plot visibilities per baseline
    - Plot visibilities vs frequency (channel 31 warants extra investigation)
- Code:
    - Output flags to FLAG data column
    - Some optimization inbuilt from using numpy and scipy but some code 
    could be refactored too
    - Add more tests
    - Create doxygen documentation
    - Add a dockerfile to run the analysis in a container